In [1]:
import numpy as np
import time
import os
from itertools import product

# Function to read benchmark files
def read_benchmark(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        n, m, k = map(int, lines[0].split())
        subsets = []
        for i in range(1, m + 1):
            subset = list(map(int, lines[i].split()[1:]))
            subsets.append(subset)
        return n, m, k, subsets

# Function to calculate fitness (coverage)
def calculate_fitness(subsets, selected):
    covered = set()
    for i in range(len(selected)):
        if selected[i] == 1:
            covered.update(subsets[i])
    return len(covered)

# PSO algorithm
def pso(benchmark_file, num_iterations, num_particles, c1, c2, w, vmax):
    n, m, k, subsets = read_benchmark(benchmark_file)
    
    # Initialize particles
    particles = np.random.randint(0, 2, size=(num_particles, m))
    for i in range(num_particles):
        # Ensure exactly k subsets are selected
        indices = np.where(particles[i] == 1)[0]
        if len(indices) > k:
            remove_indices = np.random.choice(indices, size=len(indices) - k, replace=False)
            particles[i, remove_indices] = 0
        elif len(indices) < k:
            add_indices = np.random.choice(np.where(particles[i] == 0)[0], size=k - len(indices), replace=False)
            particles[i, add_indices] = 1
    
    # Initialize velocities
    velocities = np.zeros((num_particles, m))
    
    # Initialize personal bests and global best
    personal_bests = particles.copy()
    global_best = particles[0].copy()
    global_best_fitness = calculate_fitness(subsets, global_best)
    
    best_fitness_history = []
    early_stopping_counter = 0
    max_early_stopping = 50  # Stop if no improvement for 50 iterations
    
    start_time = time.time()
    
    for iteration in range(num_iterations):
        for i in range(num_particles):
            # Update velocity
            for j in range(m):
                r1, r2 = np.random.rand(), np.random.rand()
                velocities[i, j] = w * velocities[i, j] + c1 * r1 * (personal_bests[i, j] - particles[i, j]) + c2 * r2 * (global_best[j] - particles[i, j])
                velocities[i, j] = np.clip(velocities[i, j], -vmax, vmax)
            
            # Update position using sigmoid function
            probabilities = 1 / (1 + np.exp(-velocities[i]))
            new_position = np.where(probabilities > np.random.rand(m), 1, 0)
            
            # Ensure exactly k subsets are selected
            indices = np.where(new_position == 1)[0]
            if len(indices) > k:
                remove_indices = np.random.choice(indices, size=len(indices) - k, replace=False)
                new_position[remove_indices] = 0
            elif len(indices) < k:
                add_indices = np.random.choice(np.where(new_position == 0)[0], size=k - len(indices), replace=False)
                new_position[add_indices] = 1
            
            # Update particle position
            particles[i] = new_position
            
            # Calculate fitness
            fitness = calculate_fitness(subsets, particles[i])
            
            # Update personal best
            if fitness > calculate_fitness(subsets, personal_bests[i]):
                personal_bests[i] = particles[i].copy()
            
            # Update global best
            if fitness > global_best_fitness:
                global_best = particles[i].copy()
                global_best_fitness = fitness
                early_stopping_counter = 0
            
            best_fitness_history.append(global_best_fitness)
        
        # Early stopping
        if iteration > 0 and best_fitness_history[-1] == best_fitness_history[-2]:
            early_stopping_counter += 1
            if early_stopping_counter >= max_early_stopping:
                break
    
    end_time = time.time()
    
    return global_best_fitness, end_time - start_time

# Parameter tuning
parameters = {
    'num_iterations': [100, 200, 500],
    'num_particles': [20, 50, 100],
    'c1': [1.0, 1.5, 2.0],
    'c2': [1.0, 1.5, 2.0],
    'w': [0.5, 0.7, 0.9],
    'vmax': [1.0, 2.0, 3.0]
}

benchmark_files = [os.path.join('Benchmark', f) for f in os.listdir('Benchmark') if f.endswith('.txt')]

for benchmark_file in benchmark_files:
    print(f"Processing {benchmark_file}...")
    
    best_fitness = 0
    best_params = None
    best_time = float('inf')
    
    for params in product(*parameters.values()):
        num_iterations, num_particles, c1, c2, w, vmax = params
        
        fitness, execution_time = pso(benchmark_file, num_iterations, num_particles, c1, c2, w, vmax)
        
        print(f"Parameters: {params}, Fitness: {fitness}, Time: {execution_time:.2f} seconds")
        
        if fitness > best_fitness or (fitness == best_fitness and execution_time < best_time):
            best_fitness = fitness
            best_params = params
            best_time = execution_time
    
    print(f"Best Parameters for {benchmark_file}: {best_params}, Best Fitness: {best_fitness}, Best Time: {best_time:.2f} seconds\n")

Processing Benchmark\scp41.txt...


ValueError: not enough values to unpack (expected 3, got 2)